## **Task 1**

In [1]:
import sqlite3
import csv
con = sqlite3.connect('original_db.db')
cur = con.cursor()

In [2]:
table_name ='notown'
csv_file = 'no_town.csv'

with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    columns = ', '.join(header)
    cur.execute(f'CREATE TABLE {table_name} ({columns})')

# Import data from CSV into the table
with open(csv_file, 'r') as file:
    next(file)  # Skip header row
    cur.executemany(f'INSERT INTO {table_name} VALUES ({",".join(["?"] * len(header))})', csv.reader(file))

In [3]:
statement = '''
SELECT * FROM notown;
'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['num', 'street', 'street_type', 'name', 'ssn', 'album_id', 'album_name', 'date', 'album_type', 'instrument_id', 'instrument_type', 'key']
----------------
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '7', " 'F'", '2004', " 'CD'", '9', " 'flute'", " 'C'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '21', " 'X'", '2010', " 'MC'", '5', " 'synthesizer'", " 'B'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '21', " 'X'", '2010', " 'MC'", '8', " 'synthesizer'", " 'B-flat'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '21', " 'X'", '2010', " 'MC'", '9', " 'flute'", " 'C'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '24', " 'V'", '2015', " 'MC'", '6', " 'synthesizer'", " 'C'")
('1010', " 'University'", " 'Dr.'", " 'Abel'", '100000001', '5', " 'D'", '1991', " 'CD'", '3', " 'guitar'", " 'E'")
('1010', " 'University'", " 'Dr.'", " 'Abel'", '100000001', '5', " 'D'", '1991', " 'CD'", '6', " 'synthesizer'"

In [4]:
con.commit()

## **Task 2**

In [5]:
con = sqlite3.connect('new_db.db')
cur = con.cursor()

In [6]:
statement = '''
CREATE TABLE Musicians (
     SSN text,
     name text,
     number int,
     street_name text,
     street_type text,
     primary key (SSN)
);
'''

cur.execute(statement)

In [7]:
statement = '''
CREATE TABLE Album (
     album_id text,
     title text,
     copyright_date DATE,
     format text,
     primary key (album_id)
);
'''

cur.execute(statement)

In [8]:
statement = '''
CREATE TABLE Instrument (
     id text,
     instrument_name text,
     musical_key text,
     primary key (id)
);
'''

cur.execute(statement)

In [9]:
statement = '''
CREATE TABLE Participate_In(
     SSN text,
     album_id text,
     primary key (SSN, album_id),
     foreign key (SSN) references Musician(SSN),
     foreign key (album_id) references Album(album_id)
);
'''

cur.execute(statement)

In [10]:
statement = '''
CREATE TABLE Used_In(
     album_id text,
     id text,
     primary key (album_id, id),
     foreign key (album_id) references Musician(album_id),
     foreign key (id) references Album(id)
);
'''

cur.execute(statement)

In [11]:
con.commit()

## **Task 3**

In [12]:
# Connect to the source database
source_conn = sqlite3.connect('original_db.db')
source_cursor = source_conn.cursor()

# Connect to the target database
target_conn = sqlite3.connect('new_db.db')
target_cursor = target_conn.cursor()


In [13]:
source_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = source_cursor.fetchall()
print(tables)


[('notown',)]


In [14]:
# Read data from the source table
source_cursor.execute('SELECT * FROM notown')
rows = source_cursor.fetchall()


In [15]:
# Read data from the source table
source_cursor.execute('SELECT DISTINCT SSN, name, num, street, street_type FROM notown')
rows = source_cursor.fetchall()

# Insert data into the target table
for row in rows:
    target_cursor.execute('INSERT INTO Musicians (SSN, name, number, street_name, street_type) VALUES (?, ?, ?, ?, ?)', row)

In [16]:
# Read data from the source table
source_cursor.execute('SELECT DISTINCT instrument_id, instrument_type, key FROM notown')
rows = source_cursor.fetchall()

# Insert data into the target table
for row in rows:
    target_cursor.execute('INSERT INTO Instrument (id, instrument_name, musical_key) VALUES (?, ?, ?)', row)

In [17]:
# Read data from the source table
source_cursor.execute('SELECT DISTINCT album_id, album_name, date, album_type FROM notown')
rows = source_cursor.fetchall()

# Insert data into the target table
for row in rows:
    target_cursor.execute('INSERT INTO Album (album_id, title, copyright_date, format) VALUES (?, ?, ?, ?)', row)

In [18]:
# Read data from the source table
source_cursor.execute('SELECT DISTINCT ssn, album_id FROM notown')
rows = source_cursor.fetchall()

# Insert data into the target table
for row in rows:
    target_cursor.execute('INSERT INTO Participate_In(SSN, album_id) VALUES (?, ?)', row)

In [19]:
# Read data from the source table
source_cursor.execute('SELECT DISTINCT album_id, instrument_id FROM notown')
rows = source_cursor.fetchall()

# Insert data into the target table
for row in rows:
    target_cursor.execute('INSERT INTO Used_In(album_id, id) VALUES (?, ?)', row)

In [20]:
statement = '''
SELECT * FROM notown;
'''
res = source_cursor.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['num', 'street', 'street_type', 'name', 'ssn', 'album_id', 'album_name', 'date', 'album_type', 'instrument_id', 'instrument_type', 'key']
----------------
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '7', " 'F'", '2004', " 'CD'", '9', " 'flute'", " 'C'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '21', " 'X'", '2010', " 'MC'", '5', " 'synthesizer'", " 'B'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '21', " 'X'", '2010', " 'MC'", '8', " 'synthesizer'", " 'B-flat'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '21', " 'X'", '2010', " 'MC'", '9', " 'flute'", " 'C'")
('1000', " 'University'", " 'Dr.'", " 'Antonio'", '100000000', '24', " 'V'", '2015', " 'MC'", '6', " 'synthesizer'", " 'C'")
('1010', " 'University'", " 'Dr.'", " 'Abel'", '100000001', '5', " 'D'", '1991', " 'CD'", '3', " 'guitar'", " 'E'")
('1010', " 'University'", " 'Dr.'", " 'Abel'", '100000001', '5', " 'D'", '1991', " 'CD'", '6', " 'synthesizer'"

In [21]:
statement = '''
SELECT * FROM Musicians;
'''
res = target_cursor.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['SSN', 'name', 'number', 'street_name', 'street_type']
----------------
('100000000', " 'Antonio'", 1000, " 'University'", " 'Dr.'")
('100000001', " 'Abel'", 1010, " 'University'", " 'Dr.'")
('100000002', " 'Alex'", 1020, " 'University'", " 'Dr.'")
('100000003', " 'Tristan'", 1030, " 'University'", " 'Dr.'")
('100000004', " 'Graham'", 1040, " 'University'", " 'Dr.'")
('100000005', " 'Zayden'", 1050, " 'University'", " 'Dr.'")
('100000006', " 'Judah'", 1060, " 'University'", " 'Dr.'")
('100000007', " 'Ace'", 1070, " 'University'", " 'Dr.'")
('100000008', " 'Antonio'", 1080, " 'University'", " 'Dr.'")
('100000009', " 'Abraham'", 1090, " 'University'", " 'Dr.'")
('100000010', " 'Nicolas'", 1100, " 'University'", " 'Dr.'")
('100000011', " 'Antonio'", 1110, " 'University'", " 'Dr.'")
('100000012', " 'Charlie'", 1120, " 'University'", " 'Dr.'")
('100000013', " 'Patrick'", 1130, " 'University'", " 'Dr.'")
('100000014', " 'Walker'", 1140, " 'University'", " 'Dr.'")
('100000015', " 'Joel'", 11

In [22]:
statement = '''
SELECT * FROM Instrument;
'''
res = target_cursor.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['id', 'instrument_name', 'musical_key']
----------------
('9', " 'flute'", " 'C'")
('5', " 'synthesizer'", " 'B'")
('8', " 'synthesizer'", " 'B-flat'")
('6', " 'synthesizer'", " 'C'")
('3', " 'guitar'", " 'E'")
('12', " 'guitar'", " 'B'")
('2', " 'guitar'", " 'C'")
('4', " 'guitar'", " 'B-flat'")
('7', " 'synthesizer'", " 'E'")
('1', " 'guitar'", " 'B'")
('10', " 'flute'", " 'B-flat'")
('11', " 'guitar'", " 'E'")


In [23]:
statement = '''
SELECT * FROM Album;
'''
res = target_cursor.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['album_id', 'title', 'copyright_date', 'format']
----------------
('7', " 'F'", 2004, " 'CD'")
('21', " 'X'", 2010, " 'MC'")
('24', " 'V'", 2015, " 'MC'")
('5', " 'D'", 1991, " 'CD'")
('16', " 'P'", 1995, " 'MC'")
('9', " 'I'", 2004, " 'CD'")
('14', " 'M'", 1995, " 'CD'")
('15', " 'O'", 1995, " 'CD'")
('2', " 'A'", 1991, " 'CD'")
('8', " 'H'", 2004, " 'CD'")
('10', " 'J'", 2004, " 'CD'")
('19', " 'S'", 2010, " 'MC'")
('1', " 'Z'", 1991, " 'CD'")
('4', " 'C'", 1991, " 'CD'")
('12', " 'L'", 1995, " 'CD'")
('17', " 'Q'", 1995, " 'MC'")
('18', " 'Y'", 1995, " 'MC'")
('3', " 'B'", 1991, " 'CD'")
('6', " 'E'", 1991, " 'CD'")
('23', " 'U'", 2015, " 'MC'")
('25', " 'W'", 2015, " 'MC'")
('20', " 'R'", 2010, " 'MC'")
('11', " 'K'", 1995, " 'CD'")
('22', " 'T'", 2015, " 'MC'")
('13', " 'N'", 1995, " 'CD'")


In [24]:
statement = '''
SELECT * FROM Participate_In;
'''
res = target_cursor.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['SSN', 'album_id']
----------------
('100000000', '7')
('100000000', '21')
('100000000', '24')
('100000001', '5')
('100000001', '7')
('100000001', '16')
('100000002', '9')
('100000002', '14')
('100000002', '15')
('100000002', '21')
('100000003', '2')
('100000003', '8')
('100000003', '10')
('100000003', '14')
('100000003', '15')
('100000003', '19')
('100000004', '1')
('100000004', '4')
('100000004', '5')
('100000004', '9')
('100000004', '12')
('100000004', '15')
('100000004', '24')
('100000005', '5')
('100000005', '7')
('100000005', '8')
('100000005', '14')
('100000005', '16')
('100000005', '17')
('100000005', '18')
('100000006', '3')
('100000006', '4')
('100000006', '6')
('100000006', '12')
('100000006', '21')
('100000006', '23')
('100000007', '2')
('100000007', '3')
('100000007', '12')
('100000007', '25')
('100000008', '20')
('100000009', '3')
('100000009', '7')
('100000009', '10')
('100000009', '11')
('100000009', '21')
('100000009', '22')
('100000009', '23')
('100000010', '4')
('10

In [25]:
statement = '''
SELECT * FROM Used_In;
'''
res = target_cursor.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['album_id', 'id']
----------------
('7', '9')
('21', '5')
('21', '8')
('21', '9')
('24', '6')
('5', '3')
('5', '6')
('16', '3')
('16', '5')
('16', '8')
('9', '6')
('9', '12')
('14', '3')
('14', '5')
('14', '9')
('15', '2')
('15', '12')
('2', '4')
('2', '8')
('8', '6')
('10', '4')
('10', '8')
('10', '9')
('19', '4')
('1', '6')
('1', '7')
('4', '2')
('12', '2')
('17', '2')
('17', '6')
('18', '3')
('18', '5')
('3', '1')
('3', '12')
('6', '12')
('23', '4')
('23', '8')
('23', '10')
('25', '2')
('20', '4')
('20', '10')
('11', '8')
('11', '9')
('22', '11')
('13', '1')
('13', '3')
('13', '7')


In [26]:
# Commit the changes and close connections
target_conn.commit()
target_conn.close()
source_conn.close()

## **Task 4: Summary**

In [27]:
con = sqlite3.connect('new_db.db')
cur = con.cursor()

1. A total number of musicians and a list of musicians (name and ssn).

In [28]:
statement = '''

SELECT COUNT(*) FROM Musicians;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['COUNT(*)']
----------------
(44,)


In [29]:
statement = '''

SELECT name, ssn FROM Musicians;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['name', 'SSN']
----------------
(" 'Antonio'", '100000000')
(" 'Abel'", '100000001')
(" 'Alex'", '100000002')
(" 'Tristan'", '100000003')
(" 'Graham'", '100000004')
(" 'Zayden'", '100000005')
(" 'Judah'", '100000006')
(" 'Ace'", '100000007')
(" 'Antonio'", '100000008')
(" 'Abraham'", '100000009')
(" 'Nicolas'", '100000010')
(" 'Antonio'", '100000011')
(" 'Charlie'", '100000012')
(" 'Patrick'", '100000013')
(" 'Walker'", '100000014')
(" 'Joel'", '100000015')
(" 'Olivia'", '100000016')
(" 'Emma'", '100000017')
(" 'Ava'", '100000018')
(" 'Charlotte'", '100000019')
(" 'Sophia'", '100000020')
(" 'Ava'", '100000021')
(" 'Isabella'", '100000022')
(" 'Mia'", '100000023')
(" 'Evelyn'", '100000024')
(" 'Harper'", '100000025')
(" 'Camila'", '100000026')
(" 'Gianna'", '100000027')
(" 'Ava'", '100000028')
(" 'Luna'", '100000029')
(" 'Ella'", '100000030')
(" 'Elizabeth'", '100000031')
(" 'Sofia'", '100000032')
(" 'Emily'", '100000033')
(" 'Avery'", '100000034')
(" 'Mila'", '100000035')
(" 'Antonio'

2. A total number of albums and a list of albums recorded at Notown (name and album id).

In [30]:
statement = '''

SELECT COUNT(*) FROM Album;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['COUNT(*)']
----------------
(25,)


In [31]:
statement = '''

SELECT title, album_id FROM Album;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['title', 'album_id']
----------------
(" 'F'", '7')
(" 'X'", '21')
(" 'V'", '24')
(" 'D'", '5')
(" 'P'", '16')
(" 'I'", '9')
(" 'M'", '14')
(" 'O'", '15')
(" 'A'", '2')
(" 'H'", '8')
(" 'J'", '10')
(" 'S'", '19')
(" 'Z'", '1')
(" 'C'", '4')
(" 'L'", '12')
(" 'Q'", '17')
(" 'Y'", '18')
(" 'B'", '3')
(" 'E'", '6')
(" 'U'", '23')
(" 'W'", '25')
(" 'R'", '20')
(" 'K'", '11')
(" 'T'", '22')
(" 'N'", '13')


3. A total number of instruments and a list of instruments at Notown (name, key, and id).

In [32]:
statement = '''

SELECT COUNT(*) FROM Instrument;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['COUNT(*)']
----------------
(12,)


In [33]:
statement = '''

SELECT instrument_name, musical_key, id FROM Instrument;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['instrument_name', 'musical_key', 'id']
----------------
(" 'flute'", " 'C'", '9')
(" 'synthesizer'", " 'B'", '5')
(" 'synthesizer'", " 'B-flat'", '8')
(" 'synthesizer'", " 'C'", '6')
(" 'guitar'", " 'E'", '3')
(" 'guitar'", " 'B'", '12')
(" 'guitar'", " 'C'", '2')
(" 'guitar'", " 'B-flat'", '4')
(" 'synthesizer'", " 'E'", '7')
(" 'guitar'", " 'B'", '1')
(" 'flute'", " 'B-flat'", '10')
(" 'guitar'", " 'E'", '11')


4. A table consists of the names of musicians and the total number of albums written by them.

In [34]:
statement = '''

SELECT name, COUNT(album_id) FROM Participate_In NATURAL JOIN Musicians GROUP BY ssn;

'''
res = cur.execute(statement)

relational_schema = [x[0] for x in res.description]
print(relational_schema)
print('----------------')
for instance in res:
    print(instance)

['name', 'COUNT(album_id)']
----------------
(" 'Antonio'", 3)
(" 'Abel'", 3)
(" 'Alex'", 4)
(" 'Tristan'", 6)
(" 'Graham'", 7)
(" 'Zayden'", 7)
(" 'Judah'", 6)
(" 'Ace'", 4)
(" 'Antonio'", 1)
(" 'Abraham'", 7)
(" 'Nicolas'", 2)
(" 'Antonio'", 2)
(" 'Charlie'", 5)
(" 'Patrick'", 4)
(" 'Walker'", 2)
(" 'Joel'", 7)
(" 'Olivia'", 4)
(" 'Emma'", 7)
(" 'Ava'", 2)
(" 'Charlotte'", 6)
(" 'Sophia'", 2)
(" 'Ava'", 7)
(" 'Isabella'", 5)
(" 'Mia'", 6)
(" 'Evelyn'", 2)
(" 'Harper'", 2)
(" 'Camila'", 2)
(" 'Gianna'", 4)
(" 'Ava'", 6)
(" 'Luna'", 5)
(" 'Ella'", 1)
(" 'Elizabeth'", 7)
(" 'Sofia'", 6)
(" 'Emily'", 7)
(" 'Avery'", 2)
(" 'Mila'", 5)
(" 'Antonio'", 6)
(" 'Eleanor'", 5)
(" 'Madison'", 2)
(" 'Layla'", 4)
(" 'Penelope'", 2)
(" 'Olivia'", 4)
(" 'Sadie'", 6)
(" 'Ariana'", 6)


In [35]:
"""#Close the connection (Do NOT run this if you plan to play with the query)"""

con.commit()
con.close()